# FM

Factorization Machines 实战。

先上公式：
$$
\hat{y}(\boldsymbol{x}) = w_0 + \sum_{i=1}^n w_i \cdot x_i + \sum_{i=1}^n \sum_{j = i+1}^n <\boldsymbol{v}_i, \boldsymbol{v}_j>x_i \cdot x_j 
$$

其中：$\boldsymbol{v}_i \in \mathbb{R}^k, \boldsymbol{V} \in \mathbb{R}^{n \times k}$，$\boldsymbol{v}_i$ 是 $\boldsymbol{V}$ 的第 $i$ 行。数据集 $D = \{ (\boldsymbol{x}^{(1)}, y^{(1)}), ... \}, \boldsymbol{x}^{(i)} \in \mathbb{R}^n$。

对 FM 的公式进行化简，主要针对 $\sum_{i=1}^n \sum_{j = i+1}^n <\boldsymbol{v}_i, \boldsymbol{v}_j>x_i \cdot x_j$：
$$
\begin{aligned}
 \sum_{i=1}^n \sum_{j = i+1}^n <\boldsymbol{v}_i, \boldsymbol{v}_j>x_i \cdot x_j &= \frac{1}{2} \sum_{i=1}^n \sum_{j=1}^n <\boldsymbol{v}_i, \boldsymbol{v}_j>x_i \cdot x_j - \frac{1}{2} \sum_{i=1}^n <\boldsymbol{v}_i, \boldsymbol{v}_i>x_i^2 \\
    &= \frac{1}{2} ( \sum_{i=1}^n \sum_{j=1}^n \sum_{f=1}^k v_{if}\cdot v_{jf} \cdot x_i \cdot x_j - \sum_{i=1}^n \sum_{f=1}^k v_{if}^2 \cdot x_i^2) \\ 
    &= \frac{1}{2} \sum_{f=1}^k ( \sum_{i=1}^n \sum_{j=1}^n v_{if}\cdot v_{jf} \cdot x_i \cdot x_j - \sum_{i=1}^n v_{if}^2 \cdot x_i^2)  \\
    &= \frac{1}{2} \sum_{f=1}^k ( (\sum_{i=1}^n v_{if} \cdot x_i)  (\sum_{j=1}^n v_{jf} \cdot x_j) - \sum_{i=1}^n v_{if}^2 \cdot x_i^2)  \\
    &= \frac{1}{2} \sum_{f=1}^k ( (\sum_{i=1}^n v_{if} \cdot x_i)^2 - \sum_{i=1}^n v_{if}^2 \cdot x_i^2)
\end{aligned}
$$

用 $l(y, \hat{y})$ 表示损失函数，则对样本 $(\boldsymbol{x}, y)$ 的损失进行求导：
$$
\frac{\partial l}{\partial \theta} = \frac{\partial l}{\partial \hat{y}} \frac{\partial \hat{y}}{\partial \theta}
$$
其中 $\theta$ 为参数，对不同的参数有：
$$
\frac{\partial \hat{y}}{\partial \theta} = 
\begin{cases}
1 & \theta = w_0 \\
x_i & \theta = w_i, 1 \leq i \leq n \\
x_i \sum_{j=1}^n v_{jf}\cdot x_j - v_{if} x_i^2 & \theta = v_{if}
\end{cases}
$$

FM 的矩阵形式，令 $\boldsymbol{X} \in \mathbb{R}^{m \times n}$ 表示 $m$ 个样本，为一批，如何用 FM 对一批数据 $\boldsymbol{X}$ 进行处理呢？
$$
\hat{\boldsymbol{Y}} = w_0 + \boldsymbol{X} * \boldsymbol{w} + \frac{1}{2} \{ [(\boldsymbol{X} * \boldsymbol{V})^2 - (\boldsymbol{X} \cdot \boldsymbol{X}) * (\boldsymbol{V} \cdot \boldsymbol{V})].sum(axis=1) \}
$$
上式中，$*$ 为矩阵乘法，$\cdot$ 为内积或标量乘法。

关于求导的矩阵推导：[FM模型理论之---矩阵形式解读](https://blog.csdn.net/csuyhb/article/details/100575149)。

In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras import regularizers, Input, layers, optimizers
from tensorflow.keras.layers import Layer, Dense, Add
from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import *
import tensorflow.keras.backend as K

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm

# Tensorflow 实现 FM

参考：[fun-rec/codes/base_models/FM.py](https://github.com/GZYZG/fun-rec/blob/master/codes/base_models/FM.py)。

In [187]:
# dense特征取对数　　sparse特征进行类别编码
def process_feat(data, dense_feats, sparse_feats):
    df = data.copy()
    # dense
    df_dense = df[dense_feats].fillna(0.0)
    for f in tqdm(dense_feats):
        df_dense[f] = df_dense[f].apply(lambda x: np.log(1 + x) if x > -1 else -1)

    # sparse
    df_sparse = df[sparse_feats].fillna('-1')
    for f in tqdm(sparse_feats):
        lbe = LabelEncoder()  
        df_sparse[f] = lbe.fit_transform(df_sparse[f])  # 将类别特征转化为类别编码

    df_sparse_arr = []
    for f in tqdm(sparse_feats):
        data_new = pd.get_dummies(df_sparse.loc[:, f].values)
        data_new.columns = [f + "_{}".format(i) for i in range(data_new.shape[1])]
        df_sparse_arr.append(data_new)

    df_new = pd.concat([df_dense] + df_sparse_arr, axis=1)
    return df_new


# FM 特征组合层
class CrossLayer(Layer):
    def __init__(self, input_dim, output_dim=10, **kwargs):
        super(CrossLayer, self).__init__(**kwargs)

        self.input_dim = input_dim
        self.output_dim = output_dim
        # 定义交叉特征的权重
        self.kernel = self.add_weight(name='kernel',
                                      shape=(self.input_dim, self.output_dim),
                                      initializer='glorot_uniform',
                                      trainable=True)

    def call(self, x):  # 对照上述公式中的二次项优化公式一起理解
        a = K.pow(K.dot(x, self.kernel), 2)
        b = K.dot(K.pow(x, 2), K.pow(self.kernel, 2))
        return 0.5 * K.mean(a - b, 1, keepdims=True)

# 继承 Model 定义 FM
class _FM(Model):
    def __init__(self, feature_dim):
        super(FM, self).__init__(name="FM")
        self.inputs = Input(feature_dim)
        self.linear = Dense(units=1,
                            kernel_regularizer=regularizers.l2(0.01), 
                            bias_regularizer=regularizers.l1(0.01))
        self.cross = CrossLayer(feature_dim)
        self.add = Add()
        self.pred = Dense(units=1, activation="sigmoid")
    
    def call(self, inputs):
        i = inputs # self.inputs(inputs)
        linear = self.linear(i)
        cross = self.cross(inputs)
        add = self.add([linear, cross])
        pred = self.pred(add)
        return pred
    

# 函数式定义FM模型
def FM(feature_dim):
    inputs = Input(shape=(feature_dim,))

    # 一阶特征
    linear = Dense(units=1,  # units 指输出的维数，不用指定输入的维数
                   kernel_regularizer=regularizers.l2(0.01),  # 针对权重的正则化
                   bias_regularizer=regularizers.l2(0.01))(inputs)  # 针对偏置的正则化

    # 二阶特征
    cross = CrossLayer(feature_dim)(inputs)
    add = Add()([linear, cross])  # 将一阶特征与二阶特征相加构建FM模型

    pred = Dense(units=1, activation="sigmoid")(add)
    model = Model(inputs=inputs, outputs=pred, name="FM_function")

    model.summary()
#     model.compile(loss='binary_crossentropy',
#                   optimizer=optimizers.Adam(),
#                   metrics=['binary_accuracy'])

    return model

In [53]:
# 读取数据
print('loading data...')
data = pd.read_csv('./data/kaggle_train.csv')

# dense 特征开头是I，sparse特征开头是C，Label是标签
cols = data.columns.values[1:]
dense_feats = [f for f in cols if f[0] == 'I']
sparse_feats = [f for f in cols if f[0] == 'C']
sparse_feats.insert(0, 'Id')

# 对dense数据和sparse数据分别处理
print('processing features')
print(f"unprocessed feats shape: {data.shape}")
feats = process_feat(data, dense_feats, sparse_feats)
print(f"processed feats shape: {feats.shape}")

# 划分训练和验证数据
x_trn, x_tst, y_trn, y_tst = train_test_split(feats, data['Label'], test_size=0.2, random_state=2022)

# 定义模型
model = FM(feats.shape[1])

# model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['binary_accuracy'])

# 训练模型
model.fit(x_trn, y_trn, epochs=5, batch_size=256, validation_data=(x_tst, y_tst))

loading data...
processing features
unprocessed feats shape: (1599, 41)


100%|█████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 296.43it/s]


processed feats shape: (1599, 12605)
Model: "FM_function"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 12605)]      0                                            
__________________________________________________________________________________________________
dense_37 (Dense)                (None, 1)            12606       input_21[0][0]                   
__________________________________________________________________________________________________
cross_layer_18 (CrossLayer)     (None, 1)            126050      input_21[0][0]                   
__________________________________________________________________________________________________
add_18 (Add)                    (None, 1)            0           dense_37[0][0]                   
                                                   

# Numpy 实现

不使用现有框架求导的话，则需要自己实现求导的过程。求导过程主要涉及到：$\frac{\partial l(y, \hat{y})}{\partial \hat{y}}$ 和 $\frac{\partial \hat{y}}{\partial \theta}$。其中 $\frac{\partial \hat{y}}{\partial \theta}$ 是固定的，但是对于不同的损失函数 $l$，$\frac{\partial l(y, \hat{y})}{\partial \hat{y}}$ 则需要给出对应的导数。

In [3]:
np.random.uniform(-4*np.sqrt(6 / (1+2)), 4*np.sqrt(6 / (1+2)), size=(1, 2))

array([[-1.74725189, -1.35465212]])

In [139]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def cross_entropy(ytrue, pred, reduce=False):
    """
    交叉熵损失函数
    """
    n = len(ytrue)
#     print(max(pred), min(pred))
    l = ytrue * np.log(pred) + (1 - ytrue) * np.log(1e-7+1 - pred)
    if reduce:
        l = l.sum() / n
    
    # 求 l 对 pred 的导数
    grad = pred - ytrue
    
    return -l, grad
    
    
class FM:
    def __init__(self, feature_dim, factor_dim=10, learning_rate=0.01, loss="cross_entropy"):
        self.feature_dim = feature_dim
        self.factor_dim = factor_dim
        self.learning_rate = learning_rate
        self.loss = loss
        
        self.init()
        
    def init(self):
        """
        创建参数
        -----------
        注意参数初始化的方式：
        - 从随机正态分布中采样
        """
        self.bias = np.random.uniform(-4*np.sqrt(3), 4*np.sqrt(3)) 
        self.linear_weight = np.random.uniform(-4*np.sqrt(6 / (self.feature_dim + 1)), 4*np.sqrt(6 / (self.feature_dim + 1)), size=(self.feature_dim, ))
        self.cross_weight = np.random.uniform(-4*np.sqrt(6 / (self.feature_dim+self.factor_dim)), 
                                              4*np.sqrt(6 / (self.feature_dim+self.factor_dim)), 
                                              size=(self.feature_dim, self.factor_dim))
    
    def _forward_batch(self, X):
        m = len(X)
        X = X.reshape((m, self.feature_dim))
        y_hat = self.bias + X @ self.linear_weight.reshape((-1,1)) + 0.5 * ( np.power(X@self.cross_weight, 2) - (X**2)@(self.cross_weight**2)).sum(axis=1, keepdims=1)
#         print(( np.power(X@self.cross_weight, 2) - (X**2)@(self.cross_weight**2)).sum(axis=1, keepdims=True).shape)
        return y_hat.flatten()
    
    def forward(self, X, use_batch=True):
        """
        完成一次前向，返回各个样本的预测值. 可以逐个样本计算，也可以一次性算完所有的。
        -----------
        X: (n_sample, n_feature)
        """
        if use_batch:
            return self._forward_batch(X)
        
        m, n = X.shape
        y_hat = np.empty(m)
        
        for i in range(m):
            x = X[i, :]
            tmp = self.bias
            tmp += (x * self.linear_weight).sum()
            for j in range(self.factor_dim):
                tmp += (np.power((self.cross_weight[:, j] * x).sum(), 2) - np.power(self.cross_weight[:, j] * x, 2).sum()) / 2
            y_hat[i] = tmp
        
        return y_hat
    
    def fit(self, X, y):
        """
        使用 X, y 完成一次参数更新，mini-BGD
        -----------
        X: (n_sample, n_feature)
        """
        m = len(X)
        X = np.array(X)
        y = np.array(y)
        y_hat = self.forward(X)
        pred = sigmoid(y_hat)
#         print(pred)
        loss, grads = cross_entropy(y, pred)
        
        # 小批量梯度下降更新参数
        self.bias = self.bias - self.learning_rate * grads.mean()
        self.linear_weight = self.linear_weight - self.learning_rate * (grads.reshape((-1,1)) * X).mean(axis=0)
#         self.cross_weight = self.cross_weight - self.learning_rate * grads.reshape((-1, 1)) * ( X.T @ (X @ self.cross_weight) - (X**2).T @ self.cross_weight )
        
        for i in range(self.factor_dim):
            v = self.cross_weight[:, i]  # (n_features, )
            tmp = (v * X).sum(axis=1)  # (n_sample, )
            tmp = tmp.reshape((-1, 1)) * X  # (n_sample, n_feature)
            tmp = grads.reshape((-1,1)) * tmp
            tmp = tmp.mean(axis=0)  # (n_features, )
            
            tmp2 = X ** 2  # (n_sample, n_feature)
            tmp2 = tmp2 * v  # (n_sample, n_feature)
            tmp2 = grads.reshape((-1,1)) * tmp2
            tmp2 = tmp2.mean(axis=0)
            
            self.cross_weight[:, i] = self.cross_weight[:, i] - self.learning_rate * (tmp - tmp2)
      
        acc = (y == (pred > 0.5)).sum() / m
        
#         print(f"loss: {loss.mean()}\tacc: {acc}")
        return loss.mean(), acc
    
    def predict(self, X, use_batch=True, logits=True):
        X = np.array(X)
        y_hat = self.forward(X, use_batch=use_batch)
        
        if logits:
            return sigmoid(y_hat)
        
        return y_hat

In [103]:
# 读取数据
print('loading data...')
data = pd.read_csv('./data/kaggle_train.csv')

# dense 特征开头是I，sparse特征开头是C，Label是标签
cols = data.columns.values[1:]
dense_feats = [f for f in cols if f[0] == 'I']
sparse_feats = [f for f in cols if f[0] == 'C']
sparse_feats.insert(0, 'Id')

# 对dense数据和sparse数据分别处理
print('processing features')
print(f"unprocessed feats shape: {data.shape}")
feats = process_feat(data, dense_feats, sparse_feats)
print(f"processed feats shape: {feats.shape}")

feats = np.array(feats)
labels = np.array(data['Label'])

# 划分训练和验证数据
x_trn, x_tst, y_trn, y_tst = train_test_split(feats, labels, test_size=0.2, random_state=2020)

loading data...
processing features
unprocessed feats shape: (1599, 41)


100%|█████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 264.46it/s]


processed feats shape: (1599, 12605)


In [140]:
model = FM(feats.shape[1])

In [185]:
epochs = 15
batch_size =128
x_trn 
m = len(x_trn)

for epoch in range(epochs):
    nb = int(np.ceil(m / batch_size))
    print(f"{'*'*30} Epoch: {epoch} {'*'*30}")
    for b in range(nb):
        x_batch = x_trn[b*batch_size: (b+1)*batch_size]
        y_batch = y_trn[b*batch_size: (b+1)*batch_size]
        loss, acc = model.fit(x_batch, y_batch)
        print(f"Step{b:>4}: train loss: {loss:^6.5f} train acc:{acc:^6.5f} {'='*25}")
    pred = model.predict(x_tst)
    pred = (pred > 0.5).astype(int)
    acc = (pred == y_tst).astype(int).mean()
    print(f"Epoch{epoch:>3}: acc: {acc}")

****************************** Epoch: 0 ******************************
Step   0: train loss: 0.46808 train acc:0.81250 =========================
Step   1: train loss: 0.57565 train acc:0.75781 =========================
Step   2: train loss: 0.50376 train acc:0.80469 =========================
Step   3: train loss: 0.45999 train acc:0.78125 =========================
Step   4: train loss: 0.45842 train acc:0.78906 =========================
Step   5: train loss: 0.57725 train acc:0.79688 =========================
Step   6: train loss: 0.46069 train acc:0.77344 =========================
Step   7: train loss: 0.40897 train acc:0.85938 =========================
Step   8: train loss: 0.51686 train acc:0.77344 =========================
Step   9: train loss: 0.52043 train acc:0.77165 =========================
Epoch  0: acc: 0.8125
****************************** Epoch: 1 ******************************
Step   0: train loss: 0.46385 train acc:0.81250 =========================
Step   1: train loss: 

In [186]:
pred = model.predict(x_tst)
pred = (pred > 0.5).astype(int)
acc = (pred == y_tst).astype(int).mean()
acc

0.8